In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'

In [13]:
credentials = json.load(open('../authorization.json'))
client_track_id = credentials['client_track_id']
client_secret = credentials['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_track_id=client_track_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../BigQuery/letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()

# Extract track track_id

In [14]:
%%bigquery track_track_ids
SELECT DISTINCT track_track_id FROM `snappy-boulder-378707.TrackGenre.Trackgenre`

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
track_track_ids.head()

,track_id
0,001CyR8xqmmpVZFiTZJ5BC
1,001eyxfoYptAWzvF4ewLrR
2,002LxxV7VTy5JkvkmmGanl
3,002YdWxaenIUL4PHiv5EB5
4,002nFT2kRsfvS0d17AwWsB


# Audio Features

In [35]:

audio_feat=[]
#len(track_track_ids)
for i in tqdm(range(0,len(track_track_ids),100)):
    if (i+100) < len(track_track_ids): 
        features = sp.audio_features(list(track_track_ids.iloc[i:(i+100)]['track_track_id']))
        #print(f"len each feat: {len(features)}")
        audio_feat += list(features)
    else:
        audio_feat += list(sp.audio_features(list(track_track_ids.iloc[i:]['track_track_id'])))
    #print(f"len each audio feat: {len(audio_feat)}")

#print(len(audio_feat))
audio_feat = list(filter(lambda x: x is not None, audio_feat))
#print(len(audio_feat))
audio_feat_df = pd.DataFrame(audio_feat)
audio_feat_df.to_csv('audio features.csv')


100%|██████████| 750/750 [01:36<00:00,  7.75it/s]


# Track info

In [40]:
tracks = []
for i in tqdm(range(0, len(track_ids), 50)):
    try:
        if (i+50) < len(track_ids): 
            info = sp.tracks(list(track_ids.iloc[i:(i+50)]['track_id']))['tracks']
        else:
            info = sp.tracks(list(track_ids.iloc[i:]['track_id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(track_ids): 
            info = sp.tracks(list(track_ids.iloc[i:(i+50)]['track_id']))['tracks']
        else:
            info = sp.tracks(list(track_ids.iloc[i:]['track_id']))['tracks']
        tracks += info
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info.csv')

100%|██████████| 1499/1499 [12:42<00:00,  1.96it/s]
